# Learning Vector Quantization applied to text classification

Learning Vector Quantization (Kohoen 1986, cf. Kohoen 1998) is a supervised classification approach. 
LVQ is a prototype-based competitive learning network that learns prototypes
by moving 'winning' prototypes into the direction of the respective data point (and moving 'loosing'
prototypes in the opposite direction). The prototypes are defined in the feature space of the input data,
which makes LVQ models more readily interpretable by domain experts than other neural network approaches.
Martín-Valdivia, Ureña-López and García-Vega (2007) applied LVQ to textual data in two different classification tasks,
namely text categorization and word sense disambiguation, and showed that it yields superior performance in comparison
to other common algorithms such as Naive Bayes, SVMs and k-NN.

Following this line of research, I will report on a experiment in text classfication using the LVQ implementation
provided by [Neurolab](https://github.com/zueve/neurolab). While LVQ exists in various improved variants, 
Ńeurolab only implements the basic version as originally proposed by Kohoen (LVQ1).

## Dataset - DBMail newsletter

https://archive.ics.uci.edu/ml/datasets/DBWorld+e-mails


## Imports

In [1]:
import numpy as np
import neurolab as nl
import pylab as pl
import csv
from random import shuffle

## Preprocessing

In [123]:
def encode_labels(labels):
    target = []
    for label in labels:
        if label == 0.0:
            target.append([1.0,0.0])
        elif  label == 1.0:
            target.append([0.0,1.0])
    return target

def decode_labels(targets):
    labels = []
    for target in targets:
        if np.array_equal(target, [1.0,0.0]):
            labels.append(0.0)
        elif np.array_equal(target, [0.0,1.0]):
            labels.append(1.0)
    return labels

def load_data(path):
    data = []
    labels = []
    feature_names = []
    with open(path) as f:
        reader = csv.reader(f, delimiter=',',)
        header = reader.next()
        feature_names = header[:-1] # header without CLASS
        for row in reader:
            row_values = map(float, row)
            label = row_values.pop()
            labels.append(label)
            data.append(row_values)
    return (data, encode_labels(labels), feature_names)

In [124]:
TRAIN_TEST_SPLIT = -14

In [125]:
original_data, original_target, feature_names = load_data('dbworld_subjects_stemmed.csv')
original_train_data = original_data[:TRAIN_TEST_SPLIT]
original_train_target = original_target[:TRAIN_TEST_SPLIT]
original_test_data = original_data[TRAIN_TEST_SPLIT:]
original_test_target = original_target[TRAIN_TEST_SPLIT:]

In [126]:
def shuffle_labeled_data(original_data, original_target):
    shuffled = zip(original_data, original_target)
    shuffle(shuffled)
    data = np.array([data for data, _ in shuffled])
    target = np.array([target for _, target in shuffled])
    return data, target

In [127]:
train_data, train_target = shuffle_labeled_data(original_train_data, original_train_target)
test_data, test_target = shuffle_labeled_data(original_test_data, original_test_target)

In [128]:
train_N, num_dimensions = train_data.shape
test_N, _ = test_data.shape
print 'train (N,d): ', train_N, num_dimensions
print 'test (N,d): ', test_N, num_dimensions


train (N,d):  50 229
test (N,d):  14 229


## Setting up LVQ 

Create network with 2 layers: d neurons (i.e. as man neurons as dimensions) in input layer (competitive) and 2 neurons (i.e. as many neurons as classes) in output layer (perceptrons with linear transition)

In [165]:
net = nl.net.newlvq(nl.tool.minmax(train_data), num_dimensions, [0.5, 0.5])
error = net.train(train_data, train_target, epochs=200, lr=0.0001, goal=-1)

Epoch: 100; Error: 0.5;
Epoch: 200; Error: 0.28;
The maximum number of train epochs is reached


In [166]:
output = net.sim(test_data)

## Evaluation

In [167]:
class_counts = [0,0]
for labels in target:
    if np.array_equal(labels, [ 1.0,  0.0]):
        class_counts[0] += 1
    else:
        class_counts[1] += 1
print 'Majority baseline: ', np.max(class_counts) / float(len(target))

Majority baseline:  0.546875


In [168]:
def evaluate(output, test_target):
    right = 0
    for i, labels in enumerate(output):
        if np.array_equal(labels, test_target[i]):
            right += 1
    print 'Accuracy: ', right / float(len(test_target))
evaluate(output, test_target)

Accuracy:  0.785714285714


In [173]:
def get_prototypes_and_labels(net, feature_names):
    # the prototypes are the weights of the second layer
    labeled_bags_of_words = []
    prototypes = net.layers[1].np['w']
    for prototype in prototypes:
        bag_of_words = []
        for index, entry in enumerate(prototype):
            if entry == 1:
                bag_of_words.append(feature_names[index])
        label = 'CFP' if decode_labels(net.sim([prototype]))[0] == 1 else 'NON-CFP'
        labeled_bags_of_words.append((label, bag_of_words))
    return labeled_bags_of_words

labeled_bags_of_words = get_prototypes_and_labels(net, feature_names)
for label, bag_of_words in labeled_bags_of_words:
    print label
    print bag_of_words

CFP
['10th', '13th', '1st', '2nd', '31st', '3rd', '5th', '6th', 'abstract', 'academ', 'access', 'acm', 'acn', 'advanc', 'aie', 'algorithm', 'analysi', 'analyt', 'antil', 'appli', 'applic', 'area', 'asia', 'assist', 'associ', 'augment', 'australia', 'autonom', 'base', 'big', 'bournemouth', 'call', 'centr', 'certif', 'ceur', 'cfp', 'chicago', 'chile', 'china', 'citi', 'cloud', 'commerc', 'commun', 'comput', 'computation', 'confer', 'constraint', 'cryptographi', 'cse', 'csiro', 'dalian', 'danms\xe2\x80\x9912', 'data', 'databas', 'ddi', 'deadlin', 'delai', 'delft', 'demo', 'deploy', 'deri', 'differenti', 'distribut', 'dmc', 'doc', 'doctor', 'dublin', 'due', 'edbt2012', 'effici', 'electron', 'energi', 'environ', 'er2012', 'event', 'exhibit', 'express', 'extend', 'extens', 'extrem', 'facullti', 'faculti', 'faster', 'fbk', 'fellow', 'fellowship', 'final', 'free', 'fulli', 'fund', 'galwai', 'gener', 'gi', 'graduat', 'hire', 'human', 'icistm2012', 'icsoc', 'ict', 'iir', 'illinoi', 'imag', 'inau

In [198]:
print net.sim([prototypes[1]])

[[ 1.  0.]]


In [199]:
print len(prototypes[0])
print prototypes[0]

242
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]


Majority baseline:  0.546875


Accuracy:  0.545454545455


In [203]:
def crossfold(data, target, k, epochs):
    
    split = len(data) / k

    for i in range(k):
        test_start = i * split
        test_end = (i+1) * split
        train_start = test_end
        train_end = len(data)

        test_data = data[test_start:test_end]
        train_data = data[train_start:train_end]
        
        test_target = target[test_start:test_end]
        train_target = target[train_start:train_end]
        
        second_train_start = 0
        second_train_end = 0
        
        print 'k: ', i

        if i > 0:
            second_train_end = test_start
            train_data = np.vstack([train_data, data[second_train_start:second_train_end]])
            train_target = np.vstack([train_target, target[second_train_start:second_train_end]])
            print 'train range: ', second_train_start, '-', second_train_end, ' | ', train_start, '-' , train_end
        else:
            print 'train range: ', train_start, '-' , train_end
            
        print 'test range: ', test_start, '-', test_end
            
        net = nl.net.newlvq(nl.tool.minmax(train_data), 242, [.5, .5])
        error = net.train(train_data, train_target, epochs=epochs, lr=0.1, goal=-1)
        
        output = net.sim(test_data)
        evaluate(output, test_target)

crossfold(data, target, 3, 200)

k:  0
train range:  21 - 64
test range:  0 - 21
Epoch: 100; Error: 0.511627906977;
Epoch: 200; Error: 0.0697674418605;
The maximum number of train epochs is reached
Accuracy:  0.809523809524
k:  1
train range:  0 - 21  |  42 - 64
test range:  21 - 42
Epoch: 100; Error: 0.395348837209;
Epoch: 200; Error: 0.395348837209;
The maximum number of train epochs is reached
Accuracy:  0.428571428571
k:  2
train range:  0 - 42  |  63 - 64
test range:  42 - 63
Epoch: 100; Error: 0.441860465116;
Epoch: 200; Error: 0.441860465116;
The maximum number of train epochs is reached
Accuracy:  0.52380952381


## Conclusion

Limited by technology choice, need for more evolved implementations

## References

Kohonen, T.: 1986, Learning vector quantization for pattern recognition, Technical Report TKK-
F-A601, Helsinki Univeristy of Technology, Espoo, Finland.

Kohonen, T.: 1998, Learning vector quantization, The handbook of brain theory and neural net-
works, MIT Press, Cambridge, MA, USA, pp. 537–540

Martín-Valdivia, M. T., Ureña-López, L. A., & García-Vega, M.: 2007, The Learning Vector Quantization Algorithm Applied to
Automatic Text Classification Tasks. Neural Netw., 20(6), 748–756. http://doi.org/10.1016/j.neunet.2006.12.005

Neurolab. Retrieved July 8, 2016, from https://github.com/zueve/neurolab